In [1]:
%AddJar https://oss.sonatype.org/content/repositories/snapshots/se/uu/it/cp/0.0.1-SNAPSHOT/cp-0.0.1-20170513.155506-6.jar

Starting download from https://oss.sonatype.org/content/repositories/snapshots/se/uu/it/cp/0.0.1-SNAPSHOT/cp-0.0.1-20170513.155506-6.jar
Finished download of cp-0.0.1-20170513.155506-6.jar


In [2]:
{
    
import scala.io.Source
    
import se.uu.it.cp.UnderlyingAlgorithm

// Parse dataset
case class IrisData(features: Seq[Double], label: Double)
val dataset = Source.fromFile("data/iris.data").getLines
    .map{ line =>
        val split = line.split(",")
        val label = split(4) match {
            case "Iris-setosa" => 0.0
            case "Iris-versicolor" => 1.0
            case "Iris-virginica" => 2.0
        }
        IrisData(Seq(split(0).toDouble, split(1).toDouble, 
            split(2).toDouble, split(3).toDouble), label)
    }
    .toSeq
    
// Split dataset
val (training,test) = dataset.splitAt(100)
val (properTraining, calibration) = dataset.splitAt(80)
    
// Define 5NN underlying algorithm
val alg = new UnderlyingAlgorithm(properTraining) {
    /* First we need to define the IrisData data point structure, Scala-CP aims 
     * to support any Scala/Java ML library, this is why this level of abstraction is 
     * necessary.
     */
    // Define how to make an IrisData from a feature sequence and a label
    def makeDataPoint(features: Seq[Double], label: Double) = IrisData(features,label)
    // Define how to get feature sequence from IrisData
    def getDataPointFeatures(dataPoint: IrisData) = dataPoint.features
    // Define how to get the label from IrisData
    def getDataPointLabel(dataPoint: IrisData) = dataPoint.label
    /* Once the data point structure is defined, we can implement a training
     * procedure and a nonconformity measure.
     */
    // Define training procedure
    def trainingProcedure(trainingSet: Seq[IrisData]) = (features: Seq[Double]) => {
      val trainByDistance = trainingSet.sortBy { point =>
        val squaredDist = getDataPointFeatures(point).zip(features).map {
          case (q, p) => math.pow(q - p, 2)
        }.reduce(_ + _)
        math.sqrt(squaredDist)
      }
      trainByDistance.take(5)
        .groupBy(getDataPointLabel(_))
        .maxBy(_._2.size)
        ._1
    }
    // Define nonconformity measure
    def nonConformityMeasure(newSample: IrisData) = {
      // Fraction of different closest neighbours
      val trainByDistance = this.properTrainingSet.sortBy { point =>
        val squaredDist = getDataPointFeatures(point)
        .zip(getDataPointFeatures(newSample)).map {
          case (q, p) => math.pow(q - p, 2)
        }.reduce(_ + _)
        math.sqrt(squaredDist)
      }
      val nOfUnconformal = trainByDistance.take(5)
        .count(getDataPointLabel(_) != getDataPointLabel(newSample))
      nOfUnconformal.toDouble / 5
    }
}
    
}

Name: Compile Error
Message: <console>:77: error: value properTrainingSet is not a member of se.uu.it.cp.UnderlyingAlgorithm[IrisData]
             val trainByDistance = this.properTrainingSet.sortBy { point =>
                                        ^
StackTrace: 